<body>
    <font size="2">Florian Schnabel, 11807470, SS 2023</font><br />
</body>

# Exercise 2 - Conduction and Convection with RC-Networks

This script aims to calculate the temperatures of an arbitrary number of nodes in a steady state. The nodes themselves are connected through conductances representing convection and thermal heat transfer {numref}`fig:RC-Networks`.

```{figure} ./Figures/RC-Networks.png
---
width: 350px
name: fig:RC-Networks
---
Conveductance network with three nodes {cite}`hagenthoft2001`
```

## Implementation

In [1]:
%reset

#Libraries
import numpy as np
import matplotlib.pyplot as plt

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
class node:
    # n summarizes the number of all nodes
    n=0
    def __init__(self, number):
        self.number = number
        node.n += 1
    

In [3]:
class convection:
    def __init__(self, fromNode, toNode, massflow):
        self.fromNode = fromNode
        self.toNode = toNode
        self.massflow = massflow

In [4]:
class conduction:
    def __init__(self, fromNode, toNode, conductance):
        self.fromNode = fromNode
        self.toNode = toNode
        self.K = conductance
        

In [5]:
class boundary:
    def __init__(self, toNode, temp, conductance):
        self.toNode = toNode
        self.temp = temp
        self.K = conductance

In [6]:
class heatSource:
    def __init__(self, toNode, power):
        self.toNode = toNode
        self.power = power

### Construction of the node, convection and conduction objects
Input Values according to the excersise description with the following units:
* [$T$]= $^\circ C$
* [$I$]= $W C$
* [$\overset{.}{M}$]= $kg s^{-1}$


In [7]:
nodes = []
nodes.append(node(1))
nodes.append(node(2))
nodes.append(node(3))

boundaries = []
boundaries.append(boundary(1,19,0.1))
boundaries.append(boundary(2,25,0.1))
boundaries.append(boundary(3,0,0.1))

convections = []
convections.append(convection(1,2,0.05))
convections.append(convection(3,2,0.05))
convections.append(convection(1,3,0))

conductions = []
conductions.append(conduction(1,2,0.1))
conductions.append(conduction(2,3,0.1))
conductions.append(conduction(1,3,0.1))

In [8]:
heatSources = []
heatSources.append(heatSource(1,10))
heatSources.append(heatSource(2,20))
heatSources.append(heatSource(3,10))

In [9]:
ca = 1000 #J/kgK

## Variant 1 - without convection

### Construction of the $K$ matrix and the boundary vector $I_0$

In [10]:
# reference number of nodes to increase readability
n = nodes[0].n

# Initialising conductivity matrix K
K = np.zeros((n,n))

# Initialising boundary vector I0
I0 = np.zeros((n))

# iterater to keep track of cell numbers
counter = 1
for node in nodes:
    #initialize empty conductivity matrix for the current layer
    
    I0i=0
    Ki=0
    
    for boundary in boundaries:
        if boundary.toNode == node.number:
            Ki += boundary.K
            I0i += boundary.K*boundary.temp
        
    for heatSource in heatSources:
        if heatSource.toNode == node.number:
            I0i += heatSource.power
            
    for conduction in conductions:
        if conduction.toNode == node.number or conduction.fromNode == node.number:
            Ki += conduction.K
            
    for conduction in conductions:
        if conduction.toNode == node.number:
            K[node.number-1, conduction.fromNode-1]= conduction.K
            K[conduction.fromNode-1, node.number-1]= conduction.K
    
    #for conduction in conductions:
    #    if conduction.fromNode == node.number:
    #        K[node.number-1, conduction.toNode-1]= conduction.K
    
    #for convection in convections:
    #    if convection.fromNode == node.number:
    #        Ki += conveductance.K + conveductance.massflow * ca
    #        K[conveductance.toNode-1 , node.number-1]= conveductance.K + conveductance.massflow * ca
    #        K[node.number-1 , node.number-1]= conveductance.K + conveductance.massflow * ca

   
    #defining the diagonal axis
    K[node.number-1, node.number-1]= - Ki
    
    #defining the I0
    I0[node.number-1]= I0i

In [11]:
print(K)
print(conductions[2].fromNode-1,conductions[2].toNode-1)
print(I0)
print(n)

[[-0.3  0.1  0.1]
 [ 0.1 -0.3  0.1]
 [ 0.1  0.1 -0.3]]
0 2
[11.9 22.5 10. ]
3


### Calculating the temperature field

In [12]:
T=np.linalg.solve(K,-I0)

In [13]:
print(T)

[140.75 167.25 136.  ]


In [14]:
#plt.xlabel("x position [m]")
#plt.ylabel("Temperature [°C]")
#plt.plot(x_pos, T_plus, 'o-', alpha=0.65)
#x_w=0
#for layer in layers:
#    plt.plot([x_w,x_w], [-5,35], color='k', alpha=0.7)
#    x_w += layer.width
#plt.plot([layer.width_sum,layer.width_sum], [-5,35], color='k', alpha=0.7)
#plt.title("Steady state temperature field in the building Component")
#a = np.array([1,-1,3])

## Variant 2 - without convection

### Alteration of the $K$ matrix and the boundary vector $I_0$

In [15]:
for node in nodes:
    for convection in convections:
        if convection.fromNode == node.number:
            K[node.number-1 , node.number-1] += -convection.massflow * ca
            K[convection.toNode-1 , node.number-1] += convection.massflow * ca

In [16]:
print(K)
print(conductions[2].fromNode-1,conductions[2].toNode-1)
print(I0)
print(n)

[[-50.3   0.1   0.1]
 [ 50.1  -0.3  50.1]
 [  0.1   0.1 -50.3]]
0 2
[11.9 22.5 10. ]
3


### Calculating the temperature field

In [17]:
T=np.linalg.solve(K,-I0)

In [18]:
print(T)

[  1.11706349 441.80357143   1.07936508]
